Download Fasta file with reads

In [1]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2023-06-29 23:32:30--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 52.84.97.208, 52.84.97.123, 52.84.97.40, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|52.84.97.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa.1’

phix.fa.1           100%[===================>]   5.40K  --.-KB/s    in 0s      

2023-06-29 23:32:31 (87.5 MB/s) - ‘phix.fa.1’ saved [5528/5528]



In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('phix.fa.1')

Naive Exact Algorithm : Alignment without reference genome

In [4]:
def naive(p, t):
    occurrences = [] #indices where 't' matches 'p'
    for i in range(len(t)-len(p) + 1):
        match = True
        for j in range(len(p)):
            if not t[i+j] == p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

In [5]:
#generate reads from random position in the given genome

import random
def generateReads(genome, numReads, readLen):
        
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

Matching the artificial reads generated

In [6]:
reads = generateReads(genome, 100, 100)

numMatched = 0 #reads that matched
for read in reads:
    matches = naive(read, genome)
    if len(matches)> 0:
        numMatched +=1
print('%d / %d reads matched exactly' % (numMatched, len(reads)))

100 / 100 reads matched exactly


Matching real reads from genome

In [7]:
# Download fastq file with 1000 reads
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2023-06-29 23:36:50--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 52.84.97.75, 52.84.97.208, 52.84.97.40, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|52.84.97.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq.1’

ERR266411_1.first10 100%[===================>] 248.42K  1.19MB/s    in 0.2s    

2023-06-29 23:36:51 (1.19 MB/s) - ‘ERR266411_1.first1000.fastq.1’ saved [254384/254384]



In [8]:
def readFastq(filename):
    sequences=[]
    qualities=[]
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() #base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities           

In [9]:
phix_reads, _ = readFastq('ERR266411_1.first1000.fastq.1')

In [10]:
numMatched = 0
n = 0          # 'n' is the total number of reads processed
for read in phix_reads:
    matches = naive(read, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome!' % (numMatched, n)) # proportion of reads that matched correctly

7 / 1000 reads matched the genome!


Matching a smaller segment of the read


Only 7 read matched the genome from previous matching
Could be due to sequencing errors
Keep in mind that some reads may be from the complementary strand!

In [11]:
numMatched = 0
n = 0
for read in phix_reads:
    read = read[:30] # Taking a piece of the read i.e 30 nucleotides instead of 100
    matches = naive(read, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome!' % (numMatched, n))


459 / 1000 reads matched the genome!


Including the reverse complement in the reads for matching

In [12]:
def reverseComplement(s):
    complement = {'A':'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N'}
    t =''
    for base in s:
        t = complement[base] + t
        return t

In [13]:
numMatched = 0
n = 0
for read in phix_reads:
    read = read[:30] # Take a piece of the read i.e 30 nucleotides instead of 100
    matches = naive(read, genome) # test match of read in forward direction of genome
    matches.extend(naive(reverseComplement(read), genome)) # add matches of forward reads to the reads of the reverse complement of the genome
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome!' % (numMatched, n))


1000 / 1000 reads matched the genome!
